<a href="https://colab.research.google.com/github/Granero0011/DS3-Predictive-Modeling-Challenge/blob/master/Project_Week_Predictive_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports

In [2]:
!pip install category_encoders

     |████████████████████████████████| 102kB 4.7MB/s 


In [0]:
import pandas as pd
import category_encoders as ce
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import numpy as np
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Loading the Databases


In [0]:
#Link to the csv files
train_url = 'https://raw.githubusercontent.com/Granero0011/DS3-Predictive-Modeling-Challenge/master/train_features.csv?token=AICIS43YN4IRBMAWFGMP2TK44MWMK'
test_url  = 'https://raw.githubusercontent.com/Granero0011/DS3-Predictive-Modeling-Challenge/master/test_features.csv?token=AICIS4ZAJL7URJFEEYG6N5C44MWPQ'
train_label_url='https://raw.githubusercontent.com/Granero0011/DS3-Predictive-Modeling-Challenge/master/train_labels.csv?token=AICIS45G2CLWT2U5BBSFGHC44MWRI'

In [0]:
#Load the csv in pandas libraries
#train-test split
X_train = pd.read_csv(train_url)
y_train = pd.read_csv(train_label_url)['status_group']
test=pd.read_csv(test_url)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, random_state=42)

In [8]:
X_train.head(10)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
35240,28252,200.0,2011-03-09,NaN,1757,Angli,34.588740,-9.786994,none,0,Lake Nyasa,Kiyombo,Iringa,11,5,Ludewa,Mlangali,75,True,GeoData Consultants Ltd,VWC,Anglic,False,2001,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
16282,49008,0.0,2011-08-21,Rudep,1664,DWE,31.739103,-8.771693,Mzee Selekete,0,Lake Rukwa,Mula,Rukwa,15,2,Sumbawanga Rural,Mwimbi,300,True,GeoData Consultants Ltd,VWC,NaN,False,1994,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
57019,20957,0.0,2011-07-28,Danida,0,Central government,33.922908,-9.499151,Kwa Baleke Mbila,0,Lake Nyasa,Mpugupugu,Mbeya,12,3,Kyela,Lusungo,0,True,GeoData Consultants Ltd,VWC,Ngamanga water supplied sch,True,0,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
30996,57627,0.0,2013-03-26,Mission,1532,Mission,34.820269,-11.108757,Shuleni,0,Lake Nyasa,Makugulu 'A',Ruvuma,10,3,Mbinga,Mikalanga,260,NaN,GeoData Consultants Ltd,WUG,Government,True,2005,gravity,gravity,gravity,wug,user-group,unknown,unknown,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
21149,63291,50.0,2011-03-19,Private Individual,-27,Da,38.900501,-6.451608,Pascal,0,Wami / Ruvu,Bong'Wa,Pwani,6,1,Bagamoyo,Magomeni,20,True,GeoData Consultants Ltd,Company,Bagamoyo wate,True,2009,ksb,submersible,submersible,private operator,commercial,pay per bucket,per bucket,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
10118,40991,0.0,2011-02-14,Plan Int,189,Karugendo,39.000121,-7.100938,Gumba Primary School 1,0,Wami / Ruvu,Gumba,Pwani,60,33,Kisarawe,Msimbu,120,True,GeoData Consultants Ltd,Private operator,NaN,False,2007,gravity,gravity,gravity,private operator,commercial,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe multiple,communal standpipe
35810,26332,0.0,2011-03-19,Private Individual,278,Msabi,36.700966,-8.085634,School,0,Rufiji,Kapollo,Morogoro,5,3,Kilombero,Kibaoni,1050,True,GeoData Consultants Ltd,NaN,NaN,True,2009,other - rope pump,rope pump,rope pump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
36356,28442,0.0,2011-07-20,Adap,0,ADAP,32.886501,-8.990287,Kilabuhi,0,Lake Rukwa,Mawanga,Mbeya,12,6,Mbozi,Msia,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,nira/tanira,nira/tanira,handpump,vwc,user-group,pay when scheme fails,on failure,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
12215,42392,0.0,2012-10-12,Rwssp,0,WEDECO,34.179978,-3.029721,Mtakuja,0,Lake Victoria,Nangare,Shinyanga,17,6,Meatu,Kisesa,0,True,GeoData Consultants Ltd,WUG,None,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
55619,6525,0.0,2013-02-04,Government Of Tanzania,1630,DWE,36.659227,-3.301084,Kwa Lekako,0,Pangani,Lekako,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,1999,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe


In [9]:
X_train.columns.tolist()

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [0]:
#We have 
X_train.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    2735
gps_height                   0
installer                 2746
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 266
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            2468
recorded_by                  0
scheme_management         2924
scheme_name              21137
permit                    2293
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [0]:
X_train.corr()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,1.000000,-0.004813,-0.007610,0.000353,-0.000989,-0.002700,-0.004760,-0.004003,-0.004171,-0.005807
amount_tsh,-0.004813,1.000000,0.072183,0.020624,-0.049422,0.002692,-0.024667,-0.021431,0.016306,0.063212
gps_height,-0.007610,0.072183,1.000000,0.148577,-0.036482,0.005240,-0.180715,-0.169137,0.136933,0.657407
longitude,0.000353,0.020624,0.148577,1.000000,-0.425312,0.021761,0.032922,0.152537,0.087707,0.396220
latitude,-0.000989,-0.049422,-0.036482,-0.425312,1.000000,0.007934,-0.219877,-0.202669,-0.021834,-0.243902
num_private,-0.002700,0.002692,0.005240,0.021761,0.007934,1.000000,-0.018063,-0.002775,0.005086,0.023199
region_code,-0.004760,-0.024667,-0.180715,0.032922,-0.219877,-0.018063,1.000000,0.681425,0.098667,0.031577
district_code,-0.004003,-0.021431,-0.169137,0.152537,-0.202669,-0.002775,0.681425,1.000000,0.066028,0.051441
population,-0.004171,0.016306,0.136933,0.087707,-0.021834,0.005086,0.098667,0.066028,1.000000,0.265296
construction_year,-0.005807,0.063212,0.657407,0.396220,-0.243902,0.023199,0.031577,0.051441,0.265296,1.000000


In [0]:
def MrClean(df):
    df_t= df
    df_t['gps_height'].replace(0.0, np.nan, inplace=True)
    df_t['population'].replace(0.0, np.nan, inplace=True)
    df_t['amount_tsh'].replace(0.0, np.nan, inplace=True)
    df_t['gps_height'].fillna(df_t.groupby(['region', 'district_code'])['gps_height'].transform('mean'), inplace=True)
    df_t['gps_height'].fillna(df_t.groupby(['region'])['gps_height'].transform('mean'), inplace=True)
    df_t['gps_height'].fillna(df_t['gps_height'].mean(), inplace=True)
    df_t['population'].fillna(df_t.groupby(['region', 'district_code'])['population'].transform('median'), inplace=True)
    df_t['population'].fillna(df_t.groupby(['region'])['population'].transform('median'), inplace=True)
    df_t['population'].fillna(df_t['population'].median(), inplace=True)
    df_t['amount_tsh'].fillna(df_t.groupby(['region', 'district_code'])['amount_tsh'].transform('median'), inplace=True)
    df_t['amount_tsh'].fillna(df_t.groupby(['region'])['amount_tsh'].transform('median'), inplace=True)
    df_t['amount_tsh'].fillna(df_t['amount_tsh'].median(), inplace=True)
    features=['amount_tsh', 'gps_height', 'population']
    scaler = MinMaxScaler(feature_range=(0,20))
    df_t[features] = scaler.fit_transform(df_t[features])
    df_t['longitude'].replace(0.0, np.nan, inplace=True)
    df_t['latitude'].replace(0.0, np.nan, inplace=True)
    df_t['construction_year'].replace(0.0, np.nan, inplace=True)
    df_t['latitude'].fillna(df_t.groupby(['region', 'district_code'])['latitude'].transform('mean'), inplace=True)
    df_t['longitude'].fillna(df_t.groupby(['region', 'district_code'])['longitude'].transform('mean'), inplace=True)
    df_t['longitude'].fillna(df_t.groupby(['region'])['longitude'].transform('mean'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['region', 'district_code'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['region'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['district_code'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t['construction_year'].median(), inplace=True)
    df_t['date_recorded'] = pd.to_datetime(df_t['date_recorded'])
    df_t['years_service'] = df_t.date_recorded.dt.year - df_t.construction_year
   
    garbage=['wpt_name','num_private','subvillage','region_code','recorded_by','management_group',
         'extraction_type_group','extraction_type_class','scheme_name','payment',
        'quality_group','quantity_group','source_type','source_class','waterpoint_type_group',
        'ward','public_meeting','permit','date_recorded','construction_year']
    df_t.drop(garbage,axis=1, inplace=True)
    df_t.waterpoint_type = df_t.waterpoint_type.str.lower()
    df_t.funder = df_t.funder.str.lower()
    df_t.basin = df_t.basin.str.lower()
    df_t.region = df_t.region.str.lower()
    df_t.source = df_t.source.str.lower()
    df_t.lga = df_t.lga.str.lower()
    df_t.management = df_t.management.str.lower()
    df_t.quantity = df_t.quantity.str.lower()
    df_t.water_quality = df_t.water_quality.str.lower()
    df_t.payment_type=df_t.payment_type.str.lower()
    df_t.extraction_type=df_t.extraction_type.str.lower()
    df_t["funder"].fillna("other", inplace=True)
    df_t["scheme_management"].fillna("other", inplace=True)
    df_t["installer"].fillna("other", inplace=True)
    
    #further spacial/location information
    #https://www.kaggle.com/c/sf-crime/discussion/18853
    
    return df_t

In [0]:
#Baseline model

majority_class = y_train.mode()
y_pred = [majority_class] * len(y_train)



In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.5430751964085297

In [0]:
#Logistic Regression
features = ['longitude', 'latitude', 'construction_year', 'amount_tsh']
model1 = LogisticRegression(solver='lbfgs', multi_class='auto')
model1.fit(X_train[features], y_train)
model1.score(X_val[features], y_val)

0.5430976430976431

In [0]:
#RandomForestClassifier
features = ['years_service', 'waterpoint_type', 'longitude', 'latitude','population','district_code','lga', 'basin','installer','water_quality',
'payment_type','region', 'source', 'quantity', 'extraction_type', 'funder', 'scheme_management', 'amount_tsh', 'management']
encoder = ce.OrdinalEncoder()
X_train =MrClean(X_train)
X_val=MrClean(X_val)
X_train_encoded = encoder.fit_transform(X_train[features])
X_val_encoded = encoder.transform(X_val[features])
model2 = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=42)
model2.fit(X_train_encoded, y_train)
model2.score(X_val_encoded, y_val)

0.8061279461279461

In [0]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'n_estimators': [1000, 2000], 
    'max_depth': [4, 8]
}

gridsearch = RandomizedSearchCV(
    RandomForestClassifier(n_jobs=-1, random_state=42), 
    param_distributions=param_distributions, 
    n_jobs=-1
)

gridsearch.fit(X_train_encoded, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs

In [0]:
gridsearch.best_score_

0.7419977553310887

In [0]:
#XGBClassifier
features = ['years_service', 'waterpoint_type', 'longitude', 'latitude','population','district_code','lga', 'basin','installer','water_quality',
'payment_type','region', 'source', 'quantity', 'extraction_type', 'funder', 'scheme_management', 'amount_tsh', 'management']
encoder = ce.OneHotEncoder()
X_train =MrClean(X_train)
X_val=MrClean(X_val)
X_train_encoded = encoder.fit_transform(X_train[features])
X_val_encoded = encoder.transform(X_val[features])
model3 = XGBClassifier(n_estimators=1000, n_jobs=-1, random_state=42)
model3.fit(X_train_encoded, y_train)
model3.score(X_val_encoded, y_val)





In [0]:
y_pred = model.predict(test[features])


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv('https://raw.githubusercontent.com/Granero0011/DS3-Predictive-Modeling-Challenge/master/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('DS3-Predictive_Modeling_Lionel_Fotso.csv', index=False)
files.download('DS3-Predictive_Modeling_Lionel_Fotso.csv')

In [0]:
test = MrClean(test)
test_encoded = encoder.transform(test[features])
y_pred = model2.predict(test_encoded)


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv('https://raw.githubusercontent.com/Granero0011/DS3-Predictive-Modeling-Challenge/master/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('DS3-Predictive_Modeling_Lionel_Fotso.csv', index=False)
files.download('DS3-Predictive_Modeling_Lionel_Fotso.csv')